In [14]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [286]:
k = i2c.readfrom_mem(0x5a, 0x07, 3)
inCrc = 0
for inData in [k[0], k[1]]:
    data = inCrc ^ inData;
    data <<= 8 
    for i in range(8):
        if (( data & 0x8000 ) != 0 ):
            data = data ^ (0x1070 << 3)
        data = data << 1
    inCrc = data >> 8
print(k, hex(inCrc), hex(k[2]))


b'\x7f9\xc8' 0xce 0xc8


In [194]:
def checkCRC(k):
    inCrc = 0
    for inData in [k[0], k[1]]:
        data = inCrc ^ inData;
        data <<= 8 
        for i in range(8):
            if (( data & 0x8000 ) != 0 ):
                data = data ^ (0x1070 << 3)
            data = data << 1
        inCrc = data >> 8
    return (inCrc & 0x0F) == (k[2]&0x0F)


In [10]:
# reading from IR device MLX90614
# from tab clockwise: G V SDA SCL
from machine import I2C, Pin
i2c = I2C(scl=Pin(15, Pin.OUT, Pin.PULL_UP), sda=Pin(4, Pin.OUT, Pin.PULL_UP), freq=100000)
print(i2c.scan())
i2c.writeto(0x5a, b"\x06")
k = i2c.readfrom(0x5a, 3)
print(k)


In [2]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
> auth (b0)
Traceback (most recent call last):
  File "main.py", line 100, in <module>
  File "main.py", line 98, in <module>
  File "/lib/uasyncI (4346) wifi: state: auth -> assoc (0)
io/core.py", line 180, in run_until_complete
  File "/lib/uasyncio/core.py", lI (4366) wifi: state: assoc -> run (10)
ine 154, in run_forever
  File "/lib/uasyncio/core.py", line 109, in run_forever
  File "main.py", line 42, in readtempsI (4376) wifi: connected with DoESLiverpool, channel 1

  File "sevensegmentdisplay.py", line 60, in encodeledstring
  File "sevensegmentdisplay.py", line 13, in swrite
KeyboardInterrupt: 
MicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32
Type "help()" for more information.
>>> 
>>> 
raw REPL; CTRL-B to exit
>Ready.


In [302]:
%sendtofile main.py

mqtt_local = "10.0.29.167"
from sevensegmentdisplay import encodeledstring
from machine import I2C, Pin
import time

i2c = I2C(scl=Pin(15, Pin.OUT, Pin.PULL_UP), sda=Pin(4, Pin.OUT, Pin.PULL_UP), freq=100000)

tempAmbient, tempIR = 0, 0
tempBad = True
tempTime = 0

def checkCRC(k, mask):  # mask (matching nibble) different in case
    inCrc = 0
    for inData in [k[0], k[1]]:
        data = inCrc ^ inData;
        data <<= 8 
        for i in range(8):
            if (( data & 0x8000 ) != 0 ):
                data = data ^ (0x1070 << 3)
            data = data << 1
        inCrc = data >> 8
    return (inCrc & mask) == (k[2]&mask)


async def readtemps():
    while True:
        global tempAmbient, tempIR, tempBad, tempTime
        try:
            ka = i2c.readfrom_mem(0x5a, 0x06, 3)
            ki = i2c.readfrom_mem(0x5a, 0x07, 3)
            bcrcgood = (checkCRC(ka, 0x0F) and checkCRC(ki, 0xF0))
            print(bcrcgood, ka, ki)
            if not bcrcgood:
                raise OSError()
            tempAmbient = (ka[0]+ka[1]*256)*0.02 - 273.15
            tempIR = (ki[0]+ki[1]*256)*0.02 - 273.15
            tempBad = False
        except OSError:
            tempBad = True
            
        if not tempBad:
            encodeledstring("%.2f%5.2f"% (tempAmbient, tempIR-tempAmbient))
        else:
            encodeledstring("bad %04d"%(tempTime%10000))
        tempTime = time.ticks_ms()
        await asyncio.sleep_ms(200)

        
from mqtt_as import MQTTClient
import uasyncio as asyncio
import asyn
from sevensegmentdisplay import encodeledstring

encodeledstring("start up")

import uasyncio as asyncio
import asyn
from mqtt_as import config

# Include any cross-project settings.
config['server'] = mqtt_local  # mqtt.local
config['ssid'] = 'DoESLiverpool'  # EDIT if you're using ESP32
config['wifi_pw'] = 'decafbad00'
config['mqttchannel'] = "whitesp"

mqttchannelA = config['mqttchannel']+'/tempambient'
mqttchannelI = config['mqttchannel']+'/tempir'
mqttchannelBad = config['mqttchannel']+'/tempoutofrange'

async def main(client):
    await client.connect()
    for i in range(10):
        encodeledstring("connect%d"%i)
        await asyncio.sleep_ms(100)
        
    n = 0
    while True:
        if not tempBad:  # no inconsistency problem on this as async
            if -10<tempAmbient<40:
                await client.publish(mqttchannelA, '%.2f'%tempAmbient, qos=1)
            else:
                await client.publish(mqttchannelBad, 'A %.2f'%tempAmbient, qos=1)
        
        if not tempBad:  # no inconsistency problem on this as async
            if -10<tempIR<40:
                await client.publish(mqttchannelI, '%.2f'%tempIR, qos=1)
            else:
                await client.publish(mqttchannelBad, 'I %.2f'%tempIR, qos=1)
        await asyncio.sleep_ms(5000)


MQTTClient.DEBUG = True  # Optional: print diagnostic messages
client = MQTTClient(config)
aloop = asyncio.get_event_loop()
aloop.create_task(readtemps())
encodeledstring("ready.")
try:
    aloop.run_until_complete(main(client))
except Exception:
    pass
client.close()  # Prevent LmacRxBlk:1 errors
encodeledstring("fail.")


Sent 102 lines (3157 bytes) to main.py.


In [137]:
import uasyncio as asyncio
aloop = asyncio.get_event_loop()
aloop.create_task(readtemps())
aloop.run_forever()

..

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 4, in <module>
  File "/lib/uasyncio/core.py", line 173, in run_forever
  File "/lib/uasyncio/__init__.py", line 69, in wait
KeyboardInterrupt: 


In [150]:
from mqtt_as import MQTTClient
import uasyncio as asyncio
import asyn
from sevensegmentdisplay import encodeledstring

encodeledstring("start up")

import uasyncio as asyncio
import asyn
from mqtt_as import config

# Include any cross-project settings.
config['server'] = "10.0.31.119"
config['ssid'] = 'DoESLiverpool'  # EDIT if you're using ESP32
config['wifi_pw'] = 'decafbad00'
config['mqttchannel'] = "whitesp"

mqttchannelA = config['mqttchannel']+'/tempambient'
mqttchannelI = config['mqttchannel']+'/tempir'
async def main(client):
    await client.connect()
    for i in range(10):
        encodeledstring("connect%d"%i)
        await asyncio.sleep_ms(100)
    n = 0
    while True:
        if not tempBad:
            await client.publish(mqttchannelA, '%.2f'%tempAmbient, qos=1)
        if not tempBad:
            await client.publish(mqttchannelI, '%.2f'%tempIR, qos=1)
        await asyncio.sleep_ms(5000)

MQTTClient.DEBUG = True  # Optional: print diagnostic messages
client = MQTTClient(config)
aloop = asyncio.get_event_loop()
aloop.create_task(readtemps())
encodeledstring("ready.")
try:
    aloop.run_until_complete(main(client))
finally:
    client.close()  # Prevent LmacRxBlk:1 errors
encodeledstring("fail.")


> auth (b0)
> assoc (0)
> run (10)
.
Checking WiFi integrity.
.Got reliable connection
Connecting to broker.
Connected to broker.
....RAM free 64528 alloc 46640
....ets Jun  8 2016 00:22:57

rst:0x1 (POWERON_RESET),boot:0x17 (SPI_FAST_FLASH_BOOT)
configsip: 0, SPIWP:0xee
clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00
mode:DIO, clock div:2
load:0x3fff0018,len:4
load:0x3fff001c,len:4436
load:0x40078000,len:0
load:0x40078000,len:11816
entry 0x4007a9fc
OSError: [Errno 2] ENOENT
MicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' 

In [141]:
client.connect()

In [151]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft reboot\r\nOSError: [Errno 2] ENOENT\r\nMicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'